# FEM Modelling for Single-Phase STEDIN Transformer

### Voltage-fed Couple Circuit Analysis

In frequency domain,
\begin{equation}
    -\nabla\cdot\Big[\frac{1}{\mu}\nabla u_z \Big] + j\omega\sigma u_z = J_0 = \frac{NI}{S}
\end{equation}
where,

$N$ = number of turns,

$S$ = cross-sectional area.

Circuit equation for an R-L equivalent network,
\begin{align}
    V & = j\omega N \psi + R_{ext}I + j\omega L_{ext} I \\
    V & = j\omega G'u + R_{ext}I + j\omega L_{ext} I
\end{align}

Combining the diffusion equation and circuit equation, 
\begin{equation}
    \begin{bmatrix}
        A & -f \\
        j\omega G^T & R_{ext} + j\omega L_{ext}
    \end{bmatrix} \begin{bmatrix}
        u \\
        I
    \end{bmatrix} = \begin{bmatrix}
        0 \\
        V
    \end{bmatrix}
\end{equation}
where, $V$ = known voltages, and

\begin{equation}
    G = f = \frac{N}{S}\times \frac{area\ of\ element}{3} \begin{bmatrix}
    1 \\ 1 \\ 1
\end{bmatrix}
\end{equation}

Note : The above formulation is only valid in case of per unit z-direction axis.

## Import Packages

In [1]:
print(" ▸ Loading Packages .... \r")
start = time_ns()
using gmsh
using LinearAlgebra
using LaTeXStrings
using Plots
using CSV
using DataFrames
include("../../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../../lib/FEM_VoltageFed_Tri_1e.jl");
using .FEM_VoltageFed_Tri_1e
include("../../../../lib/Post_Process_Frequency.jl");
using .Post_Process_Frequency
include("../../../../lib/Mesh_Data_stedin.jl");
using .Mesh_Data_stedin
include("../../../../lib/Save_VTK.jl");
using .Save_VTK
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Packages loaded ("*string(elapsed)*" seconds)                               ")

 ✓ Packages loaded (33.6 seconds)                               


## Julia code

In [2]:
gmsh.initialize()

## Read mesh from file
print(" ▸ Reading mesh file .... \r")
start = time_ns()
gmsh.open("../../../../Geometry/mesh/stedin_transformer.msh")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh file loaded ("*string(elapsed)*" seconds)                               ")

print(" ▸ Generating required mesh data .... \r")
start = time_ns()
mesh_data = get_mesh_data_tri_1e(gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Mesh data generated ("*string(elapsed)*" seconds)                               ")

print(" ▸ Setting initial parameters .... \r")
start = time_ns()
Ip = 17.54;   # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Vp = 15202.40;   # Primary peak phase voltage
Vs = 342.93;     # Secondary peak phase voltage
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Calculate turn density in the windings
Tp = Np / Awhv;
Ts = Ns / Awlv;

# Source turn density T
# One term for each of the windings, with a positive and negative part
sourcefunction(group_id) = [Tp*(1*(group_id==5))  -Tp*(1*(group_id==6))   Ts*(1*(group_id==11))  -Ts*(1*(group_id==12))];
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Source voltage V
# One term for each of the windings, with a positive and negative part
coil_voltage = [Vp; Vp; 0; 0];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
Rp = 400.655
Rs = 1.2999e-3
ext_resistance = [Rp; Rp; Rs; Rs];

# External resistance Rext
# One term for each of the windings, with a positive and negative part
ext_inductance = 1e-3.*[1; 1; 1; 1];

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 2500;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2)
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group);

# Conductivity
conductivityfunction(group_id) = 0;
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);

elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Initial parameters set ("*string(elapsed)*" seconds)                               ")

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids, coil_voltage, ext_resistance, ext_inductance);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy = post_process(mesh_data, u[1:mesh_data.nnodes], reluctivityperelement);

print(" ▸ Saving Plots .... \r")
start = time_ns()
# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "single_phase_stedin_transformer_B_plot_voltagefed", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "single_phase_stedin_transformer_H_plot_voltagefed", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "single_phase_stedin_transformer_mag_energy_plot_voltagefed", gmsh)
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ Plots saved ("*string(elapsed)*" seconds)                               ")

print(" ▸ Saving VTK file .... \r")
start = time_ns()
# Save as VTK file for Paraview visualization
save_vtk(mesh_data, imag(u[1:mesh_data.nnodes]), norm.(B), norm.(H), norm.(mag_energy), reluctivityperelement, "single_phase_stedin_transformer_voltagefed")
elapsed = round((time_ns() - start)/10^9, digits=2)
println(" ✓ VTK file saved ("*string(elapsed)*" seconds)                               ")

gmsh.finalize()

Info    : Reading '../../../../Geometry/mesh/stedin_transformer.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../../../Geometry/mesh/stedin_transformer.msh'
 ✓ Mesh file loaded (0.06 seconds)                               
 ✓ Mesh data generated (4.11 seconds)                               
 ✓ Initial parameters set (0.31 seconds)                               
 ✓ Matrices constructed (1.01 seconds)                               
 ✓ Solution computed (1.0 seconds)                               
 ✓ Post processing variables computed (1.13 seconds)                               
 ✓ Plots saved (23.34 seconds)                               
 ✓ VTK file saved (2.14 seconds)                               


### STEDIN Transformer : Magnetic field density
<img src="../img/single_phase_stedin_transformer_B_plot_voltagefed.png">

### STEDIN Transformer : Magnetic field strength
<img src="../img/single_phase_stedin_transformer_H_plot_voltagefed.png">

### STEDIN Transformer : Magnetic energy
<img src="../img/single_phase_stedin_transformer_mag_energy_plot_voltagefed.png">

In [3]:
display(maximum(norm.(B)))

3.1368553473701866

In [4]:
display(norm.(B[7051]))

1.70229147436843